In [1]:
import h5py
from io import BytesIO
from PIL import Image
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import wandb
import copy

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision
from torchvision.transforms import v2

from torchvision.models import resnet50, ResNet50_Weights

In [2]:
!pip install torcheval
from torcheval.metrics import BinaryAUROC

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.4 MB/s eta 0:00:00


In [3]:
train_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')

/tmp/ipykernel_25/127800222.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


In [4]:
# Image dataset for training

class ImageDataset(Dataset):
    def __init__(self, dataframe, hdfPath, augmentation=None):
        self.dataframe = dataframe
        self.hdf = h5py.File(hdfPath, mode="r")
        self.ids = dataframe['isic_id']
        self.targets = dataframe['target'].values
        self.augmentation = augmentation
        
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        itemId = self.ids[idx]
        image = Image.open(BytesIO(self.hdf[itemId][()]))
        image = np.array(image)
        target = self.targets[idx]
            
        if self.augmentation:
            image = self.augmentation(image=image)['image']
            
        return image, target

In [5]:
# Define the custom model to include softmax

class customResnet(torch.nn.Module):
    def __init__(self, pretrainedModel):
        super(customResnet, self).__init__()
        self.pretrainedModel = pretrainedModel
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(p=0.25),
            torch.nn.Linear(1000, 1),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.pretrainedModel(x)
        x = self.classifier(x)
        return x

In [6]:
# From 2020 winners
transforms_train_w = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.7),

    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    A.Resize(224, 224),
    A.CoarseDropout(max_height=int(224 * 0.375), max_width=int(224 * 0.375), max_holes=1, p=0.7),    
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

transforms_train = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),
    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.Resize(224, 224),   
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

transforms_test = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train = ImageDataset(train_metadata, '/kaggle/input/isic-2024-challenge/train-image.hdf5', augmentation=transforms_train)

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/blur/transforms.py:180: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [7]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead")
    device = torch.device("cpu")

GPU is available


In [8]:
EPOCHS = 10
batchSize = 128
learningRate = 0.0005

In [9]:
class_counts = train_metadata['target'].value_counts().to_dict()
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
sample_weights = [class_weights[target] for target in train_metadata['target']]

class_weights

{0: 2.4958444190422947e-06, 1: 0.002544529262086514}

In [10]:
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

trainDataloader = DataLoader(train, batch_size=batchSize, sampler=sampler)

pretrainedResnet = resnet50(weights=ResNet50_Weights.DEFAULT)
model = customResnet(pretrainedResnet)
model = model.to(device)

criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 159MB/s]


In [11]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb")
wandb.login(key=wandb_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
run = wandb.init(
    project="kaggle-skin-cancer",

    config={
    "learning_rate": learningRate,
    "architecture": "Resnet50",
    "epochs": EPOCHS,
    }
)

wandb: Currently logged in as: oskarkuuse. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240712_141646-h16bhuef
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fancy-cloud-14
wandb: ⭐️ View project at https://wandb.ai/oskarkuuse/kaggle-skin-cancer
wandb: 🚀 View run at https://wandb.ai/oskarkuuse/kaggle-skin-cancer/runs/h16bhuef


In [13]:
best_model = None
best_loss = np.inf
batch_idx = 0
metric = BinaryAUROC()
metric = metric.to(device)

for epoch in range(EPOCHS):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for images, labels in trainDataloader:
        
        labels = labels.float()
        
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images).squeeze(dim=1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        loss_val = loss.item()
        
        epoch_loss += loss_val
        epoch_acc += sum((outputs.detach().cpu() >= 0.5) == labels.cpu()).item() / batchSize
        
        metric.update(outputs, labels)
        
        wandb.log({"Batch loss": loss_val})
        
        batch_idx += 1
    
    wandb.log({"epoch": epoch, "BCEloss": epoch_loss / len(trainDataloader)})
    wandb.log({"epoch": epoch, "accuracy": epoch_acc / len(trainDataloader)})
    
    wandb.log({"epoch": epoch, "AUC": metric.compute().item()})
    metric.reset()
    
    if epoch_loss / len(trainDataloader) < best_loss:
        best_loss = epoch_loss / len(trainDataloader)
        best_model = copy.deepcopy(model.state_dict())
    
run.finish()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:        AUC ▁▇████████
wandb:    BCEloss █▃▂▂▁▁▁▁▁▁
wandb: Batch loss █▄▃▂▂▁▂▁▁▂▁▁▁▁▁▁▃▂▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▂▁▁▁▁▁
wandb:   accuracy ▁▇▇▇██████
wandb:      epoch ▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
wandb: 
wandb: Run summary:
wandb:        AUC 0.99998
wandb:    BCEloss 0.00479
wandb: Batch loss 1e-05
wandb:   accuracy 0.9982
wandb:      epoch 9
wandb: 
wandb: 🚀 View run fancy-cloud-14 at: https://wandb.ai/oskarkuuse/kaggle-skin-cancer/runs/h16bhuef
wandb: ⭐️ View project at: https://wandb.ai/oskarkuuse/kaggle-skin-cancer
wandb: Synced 4 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240712_141646-h16bhuef/logs


In [14]:
if best_model != None:
    torch.save(best_model, 'model_best_params.pt')

In [15]:
torch.save(model.state_dict(), 'model_last_params.pt')